In [40]:
import numpy as np
from scipy.stats import beta, bernoulli

Let each agent in our simulation have a feature which we will call `blue`. In this scenario, `blue` is a binary random variable that is either 0 or 1 probability $p$, where $p$ is an unknown random variable. 

That is, $X_1, \ldots, X_N$ is a random sample of $N$ Bernoulli random variables with parameter $p$, where $p$ is unkown. Let us assume that the prior distribution of $p$ is $p\sim B(\alpha_0, \beta_0)$ with prior probability density function $\xi$. If $x = (X_1 = x_1, \ldots, X_N = x_N)$ and the likelihood of observing $x$ given $p$ is $f(x_1|p)\cdots f(x_N|p) = f_N(x|p)$, then the posterior proabiity density function of $p$ after observing $x$ is
$$\xi(p|x) = \frac{f_N(x|p) * \xi(p)}{\int_0^1 f_N(x|p)\xi(p)dp}$$

We can find the updating rule for the posterior with a bit of math. Let $g_N(x) = \int_0^1 f_N(x|p)\xi(p)dp$ and consider, 
$$\xi(p|x) = \frac{f_N(x|p) * \xi(p)}{g_N(x)}$$
Evaluating $f_N(x|p)$ now,
$$f_N(x|p) = \prod_{i=1}^N p^{x_i}(1-p)^{x_i} = p^{\sum_{i=1}^Nx_i}(1-p)^{N - \sum_{i=1}^Nx_i} = p^y(1-p)^{N-y}$$
where $y = \sum_{i=1}^N x_i$. Evaluating $\xi(p)$ now, $$ \frac{\Gamma(\alpha_0 + \beta_0)}{\Gamma(\alpha_0)\Gamma(\beta_0)}p^{\alpha_0-1}(1-p)^{\beta_0-1}  $$
Putting this together, we get
$$\xi(p|x) = \frac{\Gamma(\alpha_0 + \beta_0)}{g_N(x)\Gamma(\alpha_0)\Gamma(\beta_0)}p^{\alpha_0+y-1}(1-p)^{\beta_0+N-y-1} $$
Since $g_N(x)$ is a constant term, this shows that $\xi(p|x)$ is proportional to a Beta distribution with parameters $\alpha_1 = \alpha_0 + y$ and $\beta_1 = \beta_0 + N - y$. The constant terms do work out to $\frac{\Gamma(\alpha_0 + \beta_0 + N)}{\Gamma(\alpha_0 + y)\Gamma(\beta_0+N-y)}$ but I omit the work for it.

Therefore, the posterior of $p\sim B(\alpha_1, \beta_1)$.

Now, this is how we update the posterior distribution of a parameter given observations of data. Hence, ***posterior***. But what we would like is to model behvaiors of stochastic agents depending on different values of states of the model. Therefore, the updating rule will be a function we modify and define in order to most accurately and realistically update the posterior distribution, which models the stochastic behavior of our agents.

Here we run through sum code using `scipy.stats` in order to illustrate what it could look like in an agent-based model.

In [41]:
# determine/get size of random sample
N = 1000

# true form of p
p = beta.rvs(1, 999, size=N)

# initialize alpha_0, beta_0 parameters, these are the hyperparameters of the prior of p
alpha0, beta0 = 1, 1

# print the p vector
# print(p)

Notice how the values of $p\in \mathbb{R}^{1000}$ are all ***very*** close to $0$. 

This is because our prior distribution with $\alpha_{true}=1, \beta_{true}=999$ has an expected value of $\mathbb{E}(p) = \frac{\alpha_{true}}{\alpha_{true} + \beta_{true}} = \frac{1}{1000} = 0.001$

Now, we observe the data $x = (X_1=x_1, \ldots, X_{1000}=x_{1000})$.

In [42]:
blue = bernoulli.rvs(p)
# print(blue)

First, notice that `scipy` allows us to pass in a vector of parameter values for Bernoulli random variables. That is, for each $i = 1, \ldots, 1000$, the $i^{th}$ entry of blue is a random variable $X_i\sim Bernoulli(p_i)$, where $p_i$ is the $i^{th}$ entry of $p\in \mathbb{R}^{1000}$.

Second, run the code above a few times and print out `blue` each time. Notice how it is different each time and also how it is primarily made up of $0$. This is because the $p_i$ for each $X_i$ is ***expected*** to be around $0.001$ and thus, each $X_i$ is expected to take the value of $1$ with probability around $0.001$. With $1000$ entries of `blue`, from our values, we would expect one $1$ on average, but don't be alarmed if there are more or less (maybe be alarmed if there are lots of ones!).

Great, we have a vector of `blue`. $1$ if they are blue and $0$ if they are not. Counting the number of agents that are blue is simple, we just sum across the vector. The number of not blue agents will be $1000 - y$ where $y$ is the number of blue agents.

In [43]:
num_blue = np.sum(blue)
num_bot_blue = N - num_blue

Now, we update the values of $\alpha_0, \beta_0$ and repeat!

In [44]:
alpha1, beta1 = alpha0 + num_blue, beta0 + N - num_blue

Here I implement a loop in order to illustrate this iterative process. Assume the number of iterations is $n = 200$ (don't confuse $n$ and $N$!)

In [32]:
N = 1000 # number of random variables in a random sample
n = 10000 # number of random samples
alpha0, beta0 = 1, 999 # true alpha, beta hyperparameters
a, b = 1, 1 # initial hyperparameters for initial prior of p
p = beta.rvs(alpha0, beta0, size=N)

for i in range(n):
    x = bernoulli.rvs(p)
    y = np.sum(x)
    a, b = a + y, b + N - y

In [33]:
print((alpha0, beta0))
x = bernoulli.rvs(p)
print(np.sum(x))
print(N - np.sum(x))

(8168, 9992832)
0
1000


So, it's likely that the values of $\alpha_0, \beta_0$ didn't stray too far away from the initial values. Typically, when we do this Bayesian updating, the parameter $p$ is unknown, and we are using the updating to figure it out, which is totally different from what we just did. What we just did was go in circles essentially. What ***really*** should be happening here is $p$ should have a fixed probability distribution and we update the hyperparameters of the posterior of $p$ until it converges to the true probability distribution of $p$. 

However, let's see what happens when we add in some terms that increase or decrease the hyperparameters. Feel free to mess around with these.

In [19]:
def dynamic(a, b, d, t, y, N):
    
    return max(1, a + y + d - t), max(1, b + N - y - d + t)

In [37]:
def D(d, t):
    if t < 5000:
        return d+1.1
    else:
        return d-1

In [38]:
t = 0
d = 0

N = 1000 # number of random variables in a random sample
n = 10000 # number of random samples
alpha0, beta0 = 1, 999 # initial alpha, beta hyperparameters
yab_list = [] # keeps track the number of blues per iteration

for i in range(n):
    d = D(d, i+1)
    p = beta.rvs(alpha0, beta0, size=N)
    x = bernoulli.rvs(p)
    y = np.sum(x)
    alpha0, beta0 = dynamic(alpha0, beta0, d, i+1, y, N)
    if i % 20 == 0:
        yab_list.append((y, alpha0, beta0))

In [39]:
print(alpha0, beta0)
print(yab_list)
max_yab = max(yab_list)
print(max_yab)
print(beta.mean(max_yab[1], max_yab[2]))

1 29285335.100000996
[(0, 1.1, 1998.9), (2, 46.099999999999994, 21953.9), (6, 176.10000000000034, 41823.9), (5, 392.10000000000116, 61607.9), (9, 673.1000000000007, 81326.9), (11, 1019.0999999999981, 100980.9), (12, 1481.099999999993, 120518.9), (12, 2009.0999999999863, 139990.9), (19, 2610.099999999977, 159389.9), (13, 3258.099999999966, 178741.9), (18, 3995.099999999952, 198004.9), (24, 4837.099999999936, 217162.9), (35, 5778.0999999999185, 236221.9), (26, 6797.0999999999085, 255202.9), (31, 7883.0999999999085, 274116.9), (31, 9079.099999999917, 292920.9), (34, 10314.099999999935, 311685.9), (37, 11675.099999999962, 330324.9), (27, 13053.099999999997, 348946.9), (34, 14582.100000000042, 367417.9), (34, 16162.100000000097, 385837.9), (41, 17814.10000000016, 404185.9), (32, 19531.10000000023, 422468.9), (39, 21333.100000000315, 440666.9), (65, 23187.100000000402, 458812.9), (52, 25173.100000000504, 476826.9), (51, 27241.10000000061, 494758.9), (45, 29380.100000000726, 512619.8999999999

Notice how the ***expected*** value of $p$ for the tuple that contains the max number of `blue` agents is a whopping $\approx 0.5$. If you look at the $\alpha_0, \beta_0$ values at the end of the list, you'll notice that it gives a Beta distribution that is centered very closely to $0$. Meaning the Beta distribution ***grew*** then ***shrank***!